In [10]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy as sp

cap = cv2.VideoCapture("vid.mp4")
cap.set(cv2.CAP_PROP_POS_MSEC,350000)
fgbg = cv2.createBackgroundSubtractorMOG2(500,50,False)
perFrame=False
i = 0
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
kernel1 = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))
while(1):
    i+=1
    ret, frame = cap.read()
    black = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),3), np.uint8)
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    res = cv2.bitwise_and(frame,frame,mask = fgmask)
    ret,thresh = cv2.threshold(fgmask,127,255,cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for elem in contours:
        area=cv2.contourArea(elem)
        if(area>3):
            mask = np.zeros(frame.shape[:2], dtype="uint8")
            cv2.drawContours(mask, [elem], -1, 255, -1)
            mean = cv2.mean(hsv, mask=mask)[:3]
            M = cv2.moments(elem)
            if(M["m00"]!=0 and M["m00"]!=0):
                cX = int((M["m10"] / M["m00"]))
                cY = int((M["m01"] / M["m00"]))
                cv2.putText(res,str(mean), (cX,cY), cv2.FONT_HERSHEY_SIMPLEX, 0.3, 255)
    cv2.drawContours(hsv, contours, -1, (255,255,255), 1)
    cv2.imshow('frame',res)
    
    delay = 1
    k = cv2.waitKey(delay)
    if(k!=-1):
        print (k)
    if k == 27:
        break
    if k == 112:
        cv2.imwrite('pic{:>05}.jpg'.format(i), res)
    if k == 32 or perFrame==True:
        k = cv2.waitKey(0)
        if(k==32):
            perFrame=False
            continue
        if k == 112:
            cv2.imwrite('pic{:>05}.jpg'.format(i), res)
            cv2.imwrite('picr{:>05}.jpg'.format(i), res1)
        if(k == 2555904): 
            perFrame=True
            continue
    
        
    

cap.release()
cv2.destroyAllWindows()

32
32
27
